In [12]:
%matplotlib inline
import sys
sys.path.append('../src')
from params import params as P
import numpy as np
import patch_sampling
from parallel import ParallelBatchIterator
np.random.seed(1)
P.DATA_LEVEL=1

train_generator, validation_generator = patch_sampling.prepare_sampler()

  0%|          | 0/36 [00:00<?, ?it/s]

Loading validation masks


  0%|          | 0/9 [00:00<?, ?it/s]

Done in  4.62551093102
... is done
Loading train masks


100%|██████████| 9/9 [00:01<00:00,  4.95it/s]

Done in  1.57757902145
... is done


In [13]:
P.DATA_FOLDER

'/media/diskB/guido/BreastDataset'

In [14]:
import os
#image_path = os.path.join(P.DATA_FOLDER, 'IDC_train/T10-10714-I-5-1_cropped_0.tif')
#mask_path = os.path.join(P.DATA_FOLDER, 'AllMasksMerged/T10-10714-I-5-1_cropped_0_Mask.tif')
image_path = os.path.join(P.DATA_FOLDER, 'DCIS_Validation/T14-03606-I-1_cropped_19.tif')
mask_path = os.path.join(P.DATA_FOLDER, 'AllMasksMerged/T14-03606-I-1_cropped_19_Mask.tif')




assert os.path.isfile(image_path)
assert os.path.isfile(mask_path)

[1 2]


In [17]:
sys.path.append('../src/sampler')
import wsi_sampler
import wsi_mask

reload(wsi_sampler); reload(wsi_mask)

m = wsi_mask.WSIMask(mask_path, border_distance=112, data_level=1)
print m.labels

[0 1 2]


In [18]:
import matplotlib.pyplot as plt
plt.imshow(m.image, cmap='gray')
plt.show()
print np.unique(m.image)

(189, 446)


In [64]:
reload(wsi_sampler); reload(wsi_mask)
s = wsi_sampler.WSISampler(image_path, m, 1,(224,224))
image, coord = s.sample(return_position=True)
print coord
plt.imshow(image.transpose(1,2,0))

In [43]:
full_image = s.sample_full()
plt.imshow(full_image.transpose(1,2,0))

In [ ]:
import multiresolutionimageinterface as mir
import time
from tqdm import tqdm
for x in tqdm(range(5000)):
    mr = mir.MultiResolutionImageReader()
    im = mr.open(image_path)
    im.close()
    
for x in tqdm(range(1000)):
    s.sample()

In [ ]:
import dataset
files = dataset.train_filenames()
len(files)
len(files[0]), len(files[1]), len(files[2])

files, mask_sources = dataset.per_class_filelist(files[0],files[1],files[2],dataset.mask_folder(),{},(1352,670,832))


In [ ]:
print len(files)
print len(mask_sources)

from multiprocessing.pool import ThreadPool as Pool
import util
import joblib

def create_sampler(wsi_filename):
    mask_filename = mask_sources[wsi_filename]
    mask = wsi_mask.WSIMask(mask_path, border_distance=112)
    sampler = wsi_sampler.WSISampler(image_path, mask, 0, (224,244))
    return sampler

pool = Pool(6)
samplers = list(util.pool_progress(pool, create_sampler, files))

    

In [ ]:
import cPickle as pickle

pool.close()
pool.join()

with open('/mnt/diskB/guido/samplers.pkl','wb') as f:
    pickle.dump(samplers,f, protocol=2)


In [ ]:
del samplers
del samplers_loaded


In [ ]:
import time
s = time.time()
with open('/mnt/diskB/guido/samplers.pkl','rb') as f:
    samplers_loaded = pickle.load(f)
print time.time() - s
# 165.624352932


In [ ]:
del samplers_loaded
import gzip
with gzip.open('/mnt/diskB/guido/samplers.pkl.gz','wb') as f:
    pickle.dump(samplers,f, protocol=2)
    


In [ ]:
import time
s = time.time()
with gzip.open('/mnt/diskB/guido/samplers.pkl.gz','rb') as f:
    samplers_loaded = pickle.load(f)
print time.time() - s
# 51.4443669319
